In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import time
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!unzip "/content/drive/My Drive/zwierzeta.zip" -d "/content/data"

data_dir = '/content/data/raw-img'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5177, 0.5003, 0.4126],
                         std=[0.2141, 0.2137, 0.2157])
])

dataSet = datasets.ImageFolder(root=data_dir, transform=transform)
liczba_klas = len(dataSet.classes)

rozmiarTreningowy = int(0.8 * len(dataSet))
rozmiarWalidacyjny = len(dataSet) - rozmiarTreningowy
TrainDataSet, valDataSet = random_split(dataSet, [rozmiarTreningowy, rozmiarWalidacyjny])

TrainDataLoader = DataLoader(TrainDataSet, batch_size=8, shuffle=True, num_workers=2)
ValDataLoader = DataLoader(valDataSet, batch_size=8, shuffle=False, num_workers=2)

In [ ]:
class TinyVGGInspirowanyCNN(nn.Module):
    def __init__(self, liczba_klas=10):
        super(TinyVGGInspirowanyCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 8 * 8, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(256, liczba_klas)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

model = TinyVGGInspirowanyCNN(liczba_klas=liczba_klas).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def jednaEpokaTrening(model, loader, optimizer, criterion, device):
    model.train()
    lossSuma = 0.0
    liczbaPoprawnych = 0
    liczbaPrzykladow = 0

    for obrazy, etykiety in loader:
        obrazy, etykiety = obrazy.to(device), etykiety.to(device)

        optimizer.zero_grad()
        wyjscia = model(obrazy)
        loss = criterion(wyjscia, etykiety)
        loss.backward()
        optimizer.step()

        lossSuma += loss.item() * obrazy.size(0)
        _, predykcje = torch.max(wyjscia, 1)
        liczbaPoprawnych += (predykcje == etykiety).sum().item()
        liczbaPrzykladow += etykiety.size(0)

    srLoss = lossSuma / liczbaPrzykladow
    srAccuracy = liczbaPoprawnych / liczbaPrzykladow
    return srLoss, srAccuracy

def jednaEpokaWalidacja(model, dataloader, criterion, device):
    model.eval()
    lossSuma = 0.0
    liczbaPoprawnych = 0
    liczbaPrzykladow = 0

    with torch.no_grad():
        for obrazy, etykiety in dataloader:
            obrazy, etykiety = obrazy.to(device), etykiety.to(device)
            wyjscia = model(obrazy)
            loss = criterion(wyjscia, etykiety)

            lossSuma += loss.item() * obrazy.size(0)
            _, przewidywania = torch.max(wyjscia, 1)
            liczbaPoprawnych += (przewidywania == etykiety).sum().item()
            liczbaPrzykladow += etykiety.size(0)

    srLoss = lossSuma / liczbaPrzykladow
    srAccuracy = liczbaPoprawnych / liczbaPrzykladow
    return srLoss, srAccuracy


In [ ]:
bestAccuracy = 0.0
epoki = 15

historia_train_loss = []
historia_train_acc = []
historia_val_loss = []
historia_val_acc = []

for epoka in range(epoki):
    start = time.time()

    trainLoss, trainAccuracy = jednaEpokaTrening(
        model, TrainDataLoader, optimizer, criterion, device
    )

    valLoss, valAccuracy = jednaEpokaWalidacja(
        model, ValDataLoader, criterion, device
    )

    czas_trwania = time.time() - start

    print(f"Epoka {epoka+1}/{epoki} - "
          f"Strata treningowa: {trainLoss:.4f}, Dokładność treningowa: {trainAccuracy:.4f} | "
          f"Strata walidacyjna: {valLoss:.4f}, Dokładność walidacyjna: {valAccuracy:.4f} | "
          f"Czas: {czas_trwania:.1f}s")

    # Zapisz najlepszy model
    if valAccuracy > bestAccuracy:
        bestAccuracy = valAccuracy
        torch.save(model.state_dict(), 'najlepszy_model_prostycnn.pth')
        print("Zapisano najlepszy model!")

    # Zapisywanie historii do wykresów
    historia_train_loss.append(trainLoss)
    historia_train_acc.append(trainAccuracy)
    historia_val_loss.append(valLoss)
    historia_val_acc.append(valAccuracy)

print(f"\nNajlepsza dokładność walidacji: {bestAccuracy:.4f}")

In [ ]:
epoki_zakres = range(1, epoki + 1)

plt.figure(figsize=(14, 5))

# Strata
plt.subplot(1, 2, 1)
plt.plot(epoki_zakres, historia_train_loss, label='Train Loss')
plt.plot(epoki_zakres, historia_val_loss, label='Val Loss')
plt.xlabel('Epoka')
plt.ylabel('Strata')
plt.title('Strata podczas treningu')
plt.legend()

# Dokładność
plt.subplot(1, 2, 2)
plt.plot(epoki_zakres, historia_train_acc, label='Train Accuracy')
plt.plot(epoki_zakres, historia_val_acc, label='Val Accuracy')
plt.xlabel('Epoka')
plt.ylabel('Dokładność')
plt.title('Dokładność podczas treningu')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
klasy = { 0: "pies",
            1: "koń",
            2: "słoń",
            3: "motyl",
            4: "kura",
            5: "kot",
            6: "krowa",
            7: "owca",
            8: "wąż",
            9: "wiewiórka"}

from sklearn.metrics import (
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score
)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

model.eval()
wszystkie_predykcje = []
wszystkie_etykiety = []

with torch.no_grad():
    for obrazy, etykiety in ValDataLoader:
        obrazy = obrazy.to(device)
        etykiety = etykiety.to(device)

        wyjscia = model(obrazy)
        _, predykcje = torch.max(wyjscia, 1)

        wszystkie_predykcje.extend(predykcje.cpu().tolist())
        wszystkie_etykiety.extend(etykiety.cpu().tolist())

wszystkie_predykcje = np.array(wszystkie_predykcje)
wszystkie_etykiety = np.array(wszystkie_etykiety)

# Macierz pomyłek
macierz_pomylek = confusion_matrix(wszystkie_etykiety, wszystkie_predykcje)
print("Macierz pomyłek:\n", macierz_pomylek)

# Metryki
precision = precision_score(wszystkie_etykiety, wszystkie_predykcje, average='macro')
recall = recall_score(wszystkie_etykiety, wszystkie_predykcje, average='macro')
f1 = f1_score(wszystkie_etykiety, wszystkie_predykcje, average='macro')

print(f"Precision (macro): {precision:.4f}")
print(f"Recall (macro): {recall:.4f}")
print(f"F1-score (macro): {f1:.4f}")

# Przygotowanie nazw klas do osi macierzy pomyłek
class_names = [klasy[i] for i in range(len(klasy))]

# Wizualizacja macierzy pomyłek
plt.figure(figsize=(10, 8))
sns.heatmap(macierz_pomylek, annot=True, fmt="d", cmap="Blues",
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predykcja")
plt.ylabel("Rzeczywista etykieta")
plt.title("Macierz pomyłek (Confusion Matrix)")
plt.tight_layout()
plt.show()